In [17]:
#pip install torch transformers nltk spacy
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:


# Load dataset
dataset = pd.read_csv('dataset/RAW_recipes.csv')

sampled_dataset = dataset.sample(frac=0.01, random_state=42)

# Preprocessing steps
# Tokenization and normalization example
sampled_dataset['ingredients'] = sampled_dataset['ingredients'].apply(nltk.word_tokenize)
# More preprocessing as needed

# Create text data for training
sampled_dataset['text'] = sampled_dataset.apply(lambda row: f"Ingredients: {row['ingredients']} \nRecipe: {row['steps']}", axis=1)

# Split the data
train_data, test_data = train_test_split(sampled_dataset['text'], test_size=0.2, random_state=42)

# Save train and test data to text files
train_data.to_csv('train_data.txt', index=False, header=False)
test_data.to_csv('test_data.txt', index=False, header=False)

       id                                        name  \
0  137739  arriba   baked winter squash mexican style   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   

                                               steps  \
0  ['make a choice and proceed with recipe', 'dep...   

                                          techniques  \
0  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                         ingredients  
0  ['winter squash', 'mexican seasoning', 'mixed ...  


In [22]:

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 model and tokenizer
model_name = 'gpt2-medium'  # You can also use 'gpt2-large' if you want
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Prepare your dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128,
    )

train_dataset = load_dataset('train_data.txt', tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)


/Users/matteorigat/PycharmProjects/nlp-project/.venv/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
def generate_recipe(ingredients_list):
    prompt = "Ingredients: " + ", ".join(ingredients_list) + "\nRecipe:"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)
    recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return recipe

ingredients = ["chicken", "rice", "garlic"]
print(generate_recipe(ingredients))